In [34]:
import pandas as pd
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import Lasso
from sklearn.metrics import root_mean_squared_error

def read_dataframe(path):
    df = pd.read_parquet(path)
    df["duration"] = (df.lpep_dropoff_datetime - df.lpep_pickup_datetime).dt.total_seconds() / 60
    df = df[(df.duration >= 1) & (df.duration <= 60)]
    df[["PULocationID", "DOLocationID"]] = df[["PULocationID", "DOLocationID"]].astype(str)
    return df

df_train = read_dataframe("../data/green_tripdata_2025-01.parquet")
df_val = read_dataframe("../data/green_tripdata_2025-02.parquet")

df_train["PU_DO"] = df_train["PULocationID"] + "_" + df_train["DOLocationID"]
df_val["PU_DO"] = df_val["PULocationID"] + "_" + df_val["DOLocationID"]

categorical = ["PU_DO"]
numerical = ["trip_distance"]

dv = DictVectorizer()
X_train = dv.fit_transform(df_train[categorical + numerical].to_dict(orient="records"))
X_val = dv.transform(df_val[categorical + numerical].to_dict(orient="records"))

y_train = df_train["duration"].values
y_val = df_val["duration"].values

In [35]:
import mlflow

mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment("class-nyc-taxi-experiment")

with mlflow.start_run(run_name="lasso_alpha_0.2"):
    alpha=0.1
    mlflow.log_param("alpha", alpha)
    
    model = Lasso(alpha=alpha)
    model.fit(X_train, y_train)
    
    y_pred = model.predict(X_val)
    rmse = root_mean_squared_error(y_val, y_pred)
    
    mlflow.log_metric("rmse", rmse)
    mlflow.sklearn.log_model(model, "model")
    
    print(f"Run finalizado. RMSE = {rmse:.4f}")

2025/10/16 21:12:54 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/10/16 21:13:10 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.
2025/10/16 21:13:10 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


Run finalizado. RMSE = 8.9926


## Model Registry example

In [9]:
from sklearn.ensemble import GradientBoostingRegressor, ExtraTreesRegressor
from sklearn.svm import LinearSVR
import pickle

import mlflow

mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment(experiment_name="nyc-taxi-mode-registry-example")

<Experiment: artifact_location='file:///c:/Users/Roi_f/pcd/nyc-taxi-predictions-2025/notebooks/mlruns/2', creation_time=1760498459739, experiment_id='2', last_update_time=1760498459739, lifecycle_stage='active', name='nyc-taxi-mode-registry-example', tags={}>

In [10]:
mlflow.sklearn.autolog()

In [11]:
training_dataset = mlflow.data.from_numpy(X_train.data, targets=y_train, name="green_tripdata_2025-01")
validation_dataset = mlflow.data.from_numpy(X_val.data, targets=y_val, name="green_tripdata_2025-02")

In [13]:
models = [
        {
            "model": GradientBoostingRegressor,
            "params": {"n_estimators": 100, "learning_rate": 0.3, "max_depth": 25, "random_state": 42},
        },
    
        {
            "model": ExtraTreesRegressor,
            "params": {"n_estimators": 100, "max_depth": 15, "random_state": 42},
        },
    
        {
            "model": LinearSVR,
            "params": {"C": 1.0, "epsilon": 0}, 
        },
]

In [14]:
with mlflow.start_run(run_name="Nested runs"):
    for model in models:
        
        model_class = model["model"]
        model_name = model_class.__name__
        params = model["params"]
        
        with mlflow.start_run(run_name=model_name, nested=True):
            
            ml_model = model_class(**params)
            ml_model.fit(X_train, y_train)
            
            y_pred = ml_model.predict(X_val)
            rmse = root_mean_squared_error(y_val, y_pred)
            mlflow.log_metric("rmse", rmse)
            
            !mkdir models
            with open("models/preprocessor.b", "wb") as f_out:
                pickle.dump(dv, f_out)
                
            mlflow.log_artifact("models/preprocessor.b", artifact_path="preprocessor")

2025/10/14 21:32:37 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.
2025/10/14 21:33:03 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.
Ya existe el subdirectorio o el archivo models.
c:\Users\Roi_f\PCD\nyc-taxi-predictions-2025\.venv\Lib\site-packages\sklearn\svm\_base.py:1250: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
2025/10/14 21:33:16 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.
Ya existe el subdirectorio o el archivo models.


## Registrar modelos a través de código

In [16]:
from sklearn.ensemble import RandomForestRegressor

with mlflow.start_run(run_name="RandomForestRegressor"):
    ml_model = RandomForestRegressor(
        n_estimators=100,
        max_depth=15,
        random_state=42
    )
    
    ml_model.fit(X_train, y_train)
    
    mlflow.sklearn.log_model(
        sk_model=model,
        artifact_path="model",
        registered_model_name="nyc-taxi-model"
    )
    
    y_pred = ml_model.predict(X_val)
    rmse = root_mean_squared_error(y_val, y_pred)
    
    mlflow.log_metric("rmse", rmse)
    
    with open("models/preprocessor.b", "wb") as f_out:
        pickle.dump(dv, f_out)
        
    mlflow.log_artifact("models/preprocessor.b", artifact_path="preprocessor")

2025/10/14 21:48:40 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.
2025/10/14 21:48:42 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/10/14 21:48:42 WARNING mlflow.sklearn: Model was missing function: predict. Not logging python_function flavor!
2025/10/14 21:48:47 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.
2025/10/14 21:48:47 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2025/10/14 21:48:47 INFO mlflow.store.db.utils: Creating initial MLflow database tables...
2025/10/14 21:48:47 INFO mlflow.store.db.utils: Updating database tables
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INF

In [18]:
run_id = input("ingrese el run_id")
run_uri = f"runs:/{run_id}/model"

result = mlflow.register_model(
    model_uri=run_uri,
    name="nyc-taxi-model"
)

Registered model 'nyc-taxi-model' already exists. Creating a new version of this model...
2025/10/14 21:53:11 WARNING mlflow.tracking._model_registry.fluent: Run with id e61f0527f0884d9b870e7c417e56e4ca has no artifacts at artifact path 'model', registering model based on models:/m-9c6d8591afee47f0afd1e107cb6622a7 instead
Created version '4' of model 'nyc-taxi-model'.


## Asignar alias

In [23]:
from mlflow import MlflowClient

client = MlflowClient(tracking_uri="sqlite:///mlflow.db")

In [24]:
client.create_registered_model(name="nyc-taxi-model")

MlflowException: Registered Model (name=nyc-taxi-model) already exists.

In [25]:
run_id = input("ingrese el run id")
run_uri = f"runs:/{run_id}/model"

result = client.create_model_version(
    name="nyc-taxi-model",
    source=run_uri,
    run_id=run_id
)

In [56]:
client.set_registered_model_alias(
    name="nyc-taxi-model",
    alias="challenger",
    version=1
)

client.set_registered_model_alias(
    name="nyc-taxi-model",  
    alias="champion",
    version=6
)

In [27]:
client.get_model_version_by_alias(
    name="nyc-taxi-model",
    alias="champion"
)

<ModelVersion: aliases=['champion'], creation_timestamp=1760669007585, current_stage='None', deployment_job_state=None, description=None, last_updated_timestamp=1760669007585, metrics=None, model_id=None, name='nyc-taxi-model', params=None, run_id='96bac94a659d4fd5a00f9f2c8f3b5aba', run_link=None, source='runs:/96bac94a659d4fd5a00f9f2c8f3b5aba/model', status='READY', status_message=None, tags={}, user_id=None, version=5>

In [28]:
client.delete_registered_model_alias(
    name="nyc-taxi-model",
    alias="champion"
)

In [29]:
client.update_model_version(
    name="nyc-taxi-model",
    version=1,
    description="WWWWWWWWWWWOOOOOOOOOOOOOOOOOOOOOOWWWWWWWWWWWWWWWWWW"
)

<ModelVersion: aliases=[], creation_timestamp=1760499497957, current_stage='None', deployment_job_state=None, description='WWWWWWWWWWWOOOOOOOOOOOOOOOOOOOOOOWWWWWWWWWWWWWWWWWW', last_updated_timestamp=1760670374272, metrics=None, model_id=None, name='nyc-taxi-model', params=None, run_id='96bac94a659d4fd5a00f9f2c8f3b5aba', run_link='', source='file:///c:/Users/Roi_f/pcd/nyc-taxi-predictions-2025/notebooks/mlruns/2/models/m-b0568f4ed99c4ea09854f2b37490d824/artifacts', status='READY', status_message=None, tags={}, user_id=None, version=1>

In [31]:
import mlflow.pyfunc

model_name = "nyc-taxi-model"
alias = "champion"

model_uri = f"models:/{model_name}@{alias}"

champion_version = mlflow.pyfunc.load_model(
    model_uri=model_uri
)

champion_version.predict(X_val)

MlflowException: Registered Model with name=nyc-taxi-model not found

In [32]:
def read_dataframe(filename):
    df = pd.read_parquet(filename)

    df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
    df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df


def preprocess(df, dv):
    df['PU_DO'] = df['PULocationID'] + '_' + df['DOLocationID']
    categorical = ['PU_DO']
    numerical = ['trip_distance']
    train_dicts = df[categorical + numerical].to_dict(orient='records')
    return dv.transform(train_dicts)


def test_model(name, alias, X_test, y_test):
    model = mlflow.pyfunc.load_model(f"models:/{name}@{alias}")
    y_pred = model.predict(X_test)
    return {"rmse": root_mean_squared_error(y_test, y_pred)}

In [37]:
df = read_dataframe("../data/green_tripdata_2025-03.parquet")

In [38]:
run_id = input("ingrese el run_id")

client.download_artifacts(
    run_id=run_id,
    path="preprocessor",
    dst_path="."
)

'c:\\Users\\Roi_f\\pcd\\nyc-taxi-predictions-2025\\notebooks\\preprocessor'

In [39]:
import pickle

with open("preprocessor/preprocessor.b", "rb") as f_in:
    dv = pickle.load(f_in)

In [40]:
X_test = preprocess(df, dv)

In [42]:
target = "duration"
y_test = df[target].values

In [52]:
test_model(name="nyc-taxi-model", alias="champion", X_test=X_test, y_test=y_test)

{'rmse': 9.162768346025532}

In [57]:
test_model(name="nyc-taxi-model", alias="challenger", X_test=X_test, y_test=y_test)

{'rmse': 6.01147524270201}